### 模拟银行服务完成程序代码

#### 目前，在以银行营业大厅为代表的窗口行业中大量使用排队（叫号）系统，该系统完全模拟了人群排队全过程，通过取票进队、排队等待、叫号服务等功能，代替了人们站队的辛苦。

#### 排队叫号软件的具体操作流程为：

<font color=brown size=4 face=雅黑>**顾客取服务序号**</font>
#### 当顾客抵达服务大厅时，前往放置在入口处旁的取号机，并按一下其上的相应服务按钮，取号机会自动打印出一张服务单。单上显示服务号及该服务号前面正在等待服务的人数。

<font color=brown size=4 face=雅黑>**服务员工呼叫顾客**</font>
#### 服务员工只需按一下其柜台上呼叫器的相应按钮，则顾客的服务号就会按顺序的显示在显示屏上，并发出“叮咚”和相关语音信息，提示顾客前往该窗口办事。当一位顾客办事完毕后，柜台服务员工只需按呼叫器相应键，即可自动呼叫下一位顾客。

#### 编写程序模拟上面的工作过程，主要要求如下：

####     程序运行后，当看到“请点击触摸屏获取号码：”的提示时，只要按回车键，即可显示“您的号码是：XXX，您前面有YYY位”的提示，其中XXX是所获得的服务号码，YYY是在XXX之前来到的正在等待服务的人数。
####     用多线程技术模拟服务窗口（可模拟多个），具有服务员呼叫顾客的行为，假设每个顾客服务的时间是10000ms，时间到后，显示“请XXX号到ZZZ号窗口！”的提示。其中ZZZ是即将为客户服务的窗口号。


In [6]:
import time, threading, random
lock = threading.Lock()




class Node(object):
    def __init__(self,sn):
        self.sn = sn
        self.next = None

class customerlst(object):
    def __init__(self, node=None):
        self.head = node
        self.tail = node

    
 
def bankteller(winum):    #柜员处理业务
    global customer
    global bank_to_close
    svcount=[]
        
    print("\n柜员窗口{0}线程已开始运行\n".format(threading.current_thread().name))
    while bank_to_close != "y":
        lock.acquire()
        #当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。

        #从队列中取出头部
        if customer.head != None:
            sn=customer.head.sn
            customer.head = customer.head.next
            print("请#{0}顾客到{1}号窗口".format(sn,winum))
            svcount.append(sn)
        
        lock.release()
        
        t=4000+random.randint(0,12000)
        time.sleep(t/1000)
        
    print("{0}号窗口接待的顾客有：".format(winum),end="")
    for i in svcount:
        print("#{0}".format(i),end="")
    print("，共#{0}人。".format(len(svcount)))
  
  
def lineup():  #顾客取服务序号
    global customer
    global bank_to_close
    global total
    
    while bank_to_close != "y":

        total+=1
        node= Node(total)
        
        lock.acquire()
        #当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。

        #从队列中取出尾部
        if customer.head == None:
            customer.head = node
            customer.tail = node
            print("顾客#{0}，你位于待办队列第一个".format(total)) 
        else:

            #计算当前待办人数
            counter=0
            cur=customer.head
            print("顾客，排在你前面的待办号有：",end="")
            while cur != None:
                counter+=1
                print("#{0}".format(cur.sn),end="")
                cur=cur.next

            customer.tail.next = node
            customer.tail = node
            print("，共{0}人。你的编号是#{1}。".format(counter,total))
        lock.release()
        
        t=2000+random.randint(0,400)
        time.sleep(t/1000)

            
if __name__ == "__main__":            
    customer = customerlst()
    bank_to_close = "n"
    total = 0



    lineupthrd = threading.Thread(target=lineup)
    lineupthrd.start()
    
    windowcount=4
    banktellerthrdLst=[None]*windowcount
    for i in range(windowcount):
        banktellerthrdLst[i] = threading.Thread(name=("teller"+str(i+1)), target=bankteller, args=(i + 1,))
        #banktellerthrdLst[i] = threading.Thread(target=bankteller, args=(i + 1,))也行
        
        banktellerthrdLst[i].start()



    '''不得有.join语句
    lineupthrd.join()
    banktellerthrdLst[3].join()
    '''
    '''如果有xxx.join()语句，则xxx线程加入后，其余线程暂停。待xxx线程运行结束后，其余线程才可继续运行。'''



    while bank_to_close != "y":
        time.sleep(10)
        bank_to_close=input("银行下班时间到了吗（y--时间到，n--时间未到）？") #如果时间到，则结束程序运行
        bank_to_close=bank_to_close.strip()
        bank_to_close=bank_to_close[0]
        bank_to_close=bank_to_close.lower()



顾客#1，你位于待办队列第一个

柜员窗口teller1线程已开始运行


柜员窗口teller2线程已开始运行

请#1顾客到2号窗口

柜员窗口teller3线程已开始运行


柜员窗口teller4线程已开始运行

顾客#2，你位于待办队列第一个
顾客，排在你前面的待办号有：#2，共1人。你的编号是#3。
顾客，排在你前面的待办号有：#2#3，共2人。你的编号是#4。
请#2顾客到2号窗口
顾客，排在你前面的待办号有：#3#4，共2人。你的编号是#5。
顾客，排在你前面的待办号有：#3#4#5，共3人。你的编号是#6。
请#3顾客到3号窗口
请#4顾客到1号窗口
顾客，排在你前面的待办号有：#5#6，共2人。你的编号是#7。
请#5顾客到4号窗口
顾客，排在你前面的待办号有：#6#7，共2人。你的编号是#8。
顾客，排在你前面的待办号有：#6#7#8，共3人。你的编号是#9。
请#6顾客到3号窗口
请#7顾客到1号窗口
顾客，排在你前面的待办号有：#8#9，共2人。你的编号是#10。
请#8顾客到4号窗口
顾客，排在你前面的待办号有：#9#10，共2人。你的编号是#11。
请#9顾客到2号窗口
顾客，排在你前面的待办号有：#10#11，共2人。你的编号是#12。
请#10顾客到1号窗口
请#11顾客到4号窗口
顾客，排在你前面的待办号有：#12，共1人。你的编号是#13。
顾客，排在你前面的待办号有：#12#13，共2人。你的编号是#14。
请#12顾客到3号窗口
顾客，排在你前面的待办号有：#13#14，共2人。你的编号是#15。
请#13顾客到1号窗口
请#14顾客到2号窗口
顾客，排在你前面的待办号有：#15，共1人。你的编号是#16。
请#15顾客到3号窗口
请#16顾客到2号窗口
顾客#17，你位于待办队列第一个
顾客，排在你前面的待办号有：#17，共1人。你的编号是#18。
顾客，排在你前面的待办号有：#17#18，共2人。你的编号是#19。
请#17顾客到4号窗口
请#18顾客到1号窗口
顾客，排在你前面的待办号有：#19，共1人。你的编号是#20。
请#19顾客到3号窗口
顾客，排在你前面的待办号有：#20，共1人。你的编号是#21。
顾客，排在你前面的待办号有：#20#21，共2人。你的编号是#22。
请#20顾客到3号窗口
请#21顾客到2